# 1. 데이터 및 라이브러리 불러오기

## 1) 데이터셋 병합 및 Rename

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid", font="Malgun Gothic")

df_info = pd.read_csv("../data/raw/big_data_set1_f.csv", encoding="cp949")
df_usage = pd.read_csv("../data/raw/big_data_set2_f.csv", encoding="cp949")
df_customer = pd.read_csv("../data/raw/big_data_set3_f.csv", encoding="cp949")

In [2]:
# 컬럼명 리네이밍
rename_info = {
    "ENCODED_MCT": "num",        # 가맹점ID
    "MCT_BSE_AR": "addr",        # 주소
    "MCT_NM": "name",            # 가맹점명
    "MCT_BRD_NUM": "brand_cd",   # 브랜드코드
    "MCT_SIGUNGU_NM": "sigungu", # 지역(시군구)
    "HPSN_MCT_ZCD_NM": "biz_type", # 업종
    "HPSN_MCT_BZN_CD_NM": "market", # 상권
    "ARE_D": "open",             # 개설일
    "MCT_ME_D": "close"          # 폐업일
}

rename_usage = {
    "ENCODED_MCT": "num",          # 가맹점ID
    "TA_YM": "ym",                 # 기준년월

    "MCT_OPE_MS_CN": "oper_month_grp", # 운영 개월 구간
    "RC_M1_SAA": "sales_grp",      # 매출금액 구간
    "RC_M1_TO_UE_CT": "txn_cnt_grp", # 매출건수 구간
    "RC_M1_UE_CUS_CN": "cust_cnt_grp", # 유니크 고객 수 구간
    "RC_M1_AV_NP_AT": "avg_pay_grp",   # 객단가 구간
    "APV_CE_RAT": "cancel_rate_grp",   # 취소율 구간

    "DLV_SAA_RAT": "delivery_sales_ratio",  # 배달 매출 비율
    "M1_SME_RY_SAA_RAT": "rel_sales_ratio", # 업종 대비 매출 비율
    "M1_SME_RY_UE_CT": "rel_txn_ratio",     # 업종 대비 건수 비율

    "M12_SME_RY_SAA_PCE_RT": "sales_rank_industry", # 업종 내 매출 순위(%)  
    "M12_SME_BZN_SAA_PCE_RT": "sales_rank_market",  # 상권 내 매출 순위(%)  
    "M12_SME_RY_ME_MCT_RAT": "share_sales_industry",# 업종 내 해지 가맹점 비중  
    "M12_SME_BZN_ME_MCT_RAT": "share_sales_market"  # 상권 내 해지 가맹점 비중
}

rename_customer = {
    "ENCODED_MCT": "num",   # 가맹점ID
    "TA_YM": "ym",          # 기준년월

    # 성별/연령별 비율
    "M12_MAL_1020_RAT": "male_20below_ratio",
    "M12_MAL_30_RAT": "male_30_ratio",
    "M12_MAL_40_RAT": "male_40_ratio",
    "M12_MAL_50_RAT": "male_50_ratio",
    "M12_MAL_60_RAT": "male_60plus_ratio",

    "M12_FME_1020_RAT": "female_20below_ratio",
    "M12_FME_30_RAT": "female_30_ratio",
    "M12_FME_40_RAT": "female_40_ratio",
    "M12_FME_50_RAT": "female_50_ratio",
    "M12_FME_60_RAT": "female_60plus_ratio",

    # 고객 행동 지표
    "MCT_UE_CLN_REU_RAT": "revisit_ratio",  # 재방문 고객 비율
    "MCT_UE_CLN_NEW_RAT": "new_ratio",      # 신규 고객 비율

    # 고객 유형 비율
    "RC_M1_SHC_RSD_UE_CLN_RAT": "resident_ratio", # 거주 인구 고객 비율
    "RC_M1_SHC_WP_UE_CLN_RAT": "worker_ratio",    # 직장인 고객 비율
    "RC_M1_SHC_FLP_UE_CLN_RAT": "floating_ratio"  # 유동인구 고객 비율
}


In [3]:

# 1. 리네이밍 적용
df_info.rename(columns=rename_info, inplace=True)
df_usage.rename(columns=rename_usage, inplace=True)
df_customer.rename(columns=rename_customer, inplace=True)


# 2. 날짜 변환
for col in ["open", "close"]:
    df_info[col] = (
        df_info[col]
        .astype(str)                              # 숫자 → 문자열
        .str.replace(r"\D", "", regex=True)       # 숫자 외 문자 제거
        .str.slice(0, 8)                          # 앞 8자리만 사용 (YYYYMMDD)
    )
    df_info[col] = pd.to_datetime(df_info[col], format="%Y%m%d", errors="coerce")

# ym: YYYYMM 숫자 → 문자열 변환 후 파싱
df_usage["ym"]    = pd.to_datetime(df_usage["ym"].astype(str), format="%Y%m", errors="coerce")
df_customer["ym"] = pd.to_datetime(df_customer["ym"].astype(str), format="%Y%m", errors="coerce")

print("usage 기간:", df_usage["ym"].min(), "~", df_usage["ym"].max())
print("customer 기간:", df_customer["ym"].min(), "~", df_customer["ym"].max())


# 3. 월별 데이터 병합
df_monthly = pd.merge(df_usage, df_customer, on=["num", "ym"], how="outer")
print("월별 병합 크기:", df_monthly.shape)


# 4. 기본 정보 붙이기
df_final = pd.merge(df_monthly, df_info, on="num", how="left")


# 5. 폐업 여부 플래그
df_final["is_closed"] = df_final["close"].notnull().astype(int)


# 6. 특수 결측치 치환
for bad in [-999999.9, -999999, "-999999.9", "-999999"]:
    df_final.replace(bad, pd.NA, inplace=True)


# 7. 숫자형 변환 (비율/랭크 등)
ratio_cols = [
    "delivery_sales_ratio", "rel_sales_ratio", "rel_txn_ratio",
    "sales_rank_industry", "sales_rank_market",
    "share_sales_industry", "share_sales_market",
    "revisit_ratio", "new_ratio",
    "resident_ratio", "worker_ratio", "floating_ratio",
    "male_20below_ratio", "male_30_ratio", "male_40_ratio",
    "male_50_ratio", "male_60plus_ratio",
    "female_20below_ratio", "female_30_ratio", "female_40_ratio",
    "female_50_ratio", "female_60plus_ratio",
]
for c in ratio_cols:
    if c in df_final.columns:
        df_final[c] = pd.to_numeric(df_final[c], errors="coerce")


# 8. 범주형 변환
group_cols = [
    "oper_month_grp", "sales_grp", "txn_cnt_grp",
    "cust_cnt_grp", "avg_pay_grp", "cancel_rate_grp"
]
for c in group_cols:
    if c in df_final.columns:
        df_final[c] = df_final[c].astype("category")

# 9. 정렬 (중복 제거는 하지 않음)
df_final = df_final.sort_values(["num", "ym"]).reset_index(drop=True)


# 10. 저장
df_final.to_csv("../data/processed/df_final.csv", index=False, encoding="utf-8-sig")
print(" 최종 데이터 크기:", df_final.shape)
print(df_final.head())


usage 기간: 2023-01-01 00:00:00 ~ 2024-12-01 00:00:00
customer 기간: 2023-01-01 00:00:00 ~ 2024-12-01 00:00:00
월별 병합 크기: (86590, 30)
 최종 데이터 크기: (86590, 39)
          num         ym oper_month_grp           sales_grp  \
0  000F03E44A 2023-01-01       5_75-90%  6_90%초과(하위 10% 이하)   
1  000F03E44A 2023-02-01       5_75-90%  6_90%초과(하위 10% 이하)   
2  000F03E44A 2023-03-01       5_75-90%  6_90%초과(하위 10% 이하)   
3  000F03E44A 2023-04-01       5_75-90%  6_90%초과(하위 10% 이하)   
4  000F03E44A 2023-05-01       5_75-90%  6_90%초과(하위 10% 이하)   

          txn_cnt_grp        cust_cnt_grp         avg_pay_grp cancel_rate_grp  \
0            5_75-90%            5_75-90%            4_50-75%         1_상위1구간   
1  6_90%초과(하위 10% 이하)  6_90%초과(하위 10% 이하)  6_90%초과(하위 10% 이하)             NaN   
2  6_90%초과(하위 10% 이하)  6_90%초과(하위 10% 이하)  6_90%초과(하위 10% 이하)             NaN   
3            5_75-90%            5_75-90%            3_25-50%         1_상위1구간   
4            5_75-90%            5_75-90%  6_90%초과(하위 10% 이하)  

In [4]:
df_final.shape

(86590, 39)

## 2) 가게별 매출변수 생성

In [5]:

# 1) 업종 평균 파일 읽기 (탭 구분)
# =========================
avg_sales = pd.read_csv("../data/raw/business category mapping.txt", sep="\t", engine="python")

# 컬럼 정리: 공백 제거 → '업종','2023매출','2024매출'
avg_sales.columns = avg_sales.columns.str.replace(r"\s+", "", regex=True).str.strip()

# 숫자 콤마 제거 → float
for c in ["2023매출", "2024매출"]:
    if c in avg_sales.columns:
        avg_sales[c] = (avg_sales[c].astype(str)
                        .str.replace(",", "", regex=False)
                        .str.strip()
                        .replace({"": np.nan})
                        .astype(float))

# 업종 키(공백 제거) 생성
avg_sales["biz_type2"] = avg_sales["업종"].astype(str).str.replace(r"\s+", "", regex=True)

# wide → long (연도/평균매출)
avg_long = (avg_sales
            .melt(id_vars=["업종", "biz_type2"],
                  value_vars=[c for c in ["2023매출","2024매출"] if c in avg_sales.columns],
                  var_name="연도열", value_name="평균매출")
            .dropna(subset=["평균매출"]))
avg_long["year"] = avg_long["연도열"].str.extract(r"(\d{4})").astype(int)
avg_long = avg_long.drop(columns=["연도열"])
src_col = "biz_type2"


- 매출 데이터 불러오기

In [6]:
avg_long.head()

,업종,biz_type2,평균매출,year
0,한식,한식,24845.8,2023
1,중식 음식점업,중식음식점업,23531.8,2023
2,일식 음식점업,일식음식점업,29295.3,2023
3,서양식 음식점업,서양식음식점업,22950.3,2023
4,기타 외국식 음식점업,기타외국식음식점업,21279.4,2023


In [7]:
import pandas as pd

def clean_category(s: pd.Series) -> pd.Series:
    if s is None or s.empty:
        return pd.Series(dtype=object)
    return (s.astype("string")
              .str.strip()
              .str.replace(r"\s+", " ", regex=True))

def unify_biz_type(df_final: pd.DataFrame) -> pd.DataFrame:
    """
    df_final을 입력받아 biz_type 기반으로 업종 통합 라벨을 적용하고,
    결과를 'biz_type2' 컬럼에 저장.
    (기존 biz_type은 그대로 유지)
    """

    # --- 0) 기본 전처리 ---
    if "biz_type" not in df_final.columns:
        df_final["biz_type"] = pd.Series(dtype=object)
    df_final["biz_type"] = clean_category(df_final.get("biz_type", pd.Series(dtype=object)))

    # 프랜차이즈 여부 생성 (기존 로직 유지)
    df_final["프랜차이즈여부"] = df_final.get("브랜드구분코드", pd.Series(index=df_final.index)).notna().astype(int)

    # --- 1) 업종 통합 매핑 ---
    groups = {
        # 한식
        "한식": [
            "한식-단품요리일반", "백반/가정식", "기사식당", "한식뷔페",
            "한식-국밥/설렁탕", "한식-찌개/전골", "한식-감자탕", "한식-죽", "한정식", "한식-두부요리",
            "한식-국수/만두", "한식-냉면", "한식-육류/고기", "한식-해물/생선"
        ],
        # 중식
        "중식음식점업": ["중식당", "중식-훠궈/마라탕", "중식-딤섬/중식만두"],
        # 일식
        "일식음식점업": [
            "일식당", "이자카야", "꼬치구이",
            "일식-덮밥/돈가스", "일식-우동/소바/라면",
            "일식-샤브샤브", "일식-초밥/롤", "일식-참치회"
        ],
        # 서양/외국식
        "서양식음식점업": ["양식", "스테이크"],
        "기타외국식음식점업": ["동남아/인도음식", "기타세계요리"],
        # 패스트푸드
        "피자·햄버거·샌드위치및유사음식점업": ["피자", "햄버거", "샌드위치/토스트"],
        "치킨전문점": ["치킨"],
        "김밥및기타간이음식점업": ["분식"],
        "간이음식포장판매전문점": ["도시락", "반찬"],
        # 제과/디저트
        "제과점": ["베이커리", "도너츠", "떡/한과", "떡/한과 제조", "마카롱", "와플/크로플", "아이스크림/빙수", "탕후루"],
        # 커피/음료
        "커피전문점": ["카페", "커피전문점", "테이크아웃커피", "테마카페"],
        "기타비알코올음료점업": ["주스", "차"],
        # 주점
        "주점업": ["호프/맥주", "일반 유흥주점", "룸살롱/단란주점",
                  "요리주점", "포장마차", "민속주점", "와인바", "와인샵", "주류"],
        # 구내식당
        "기관구내식당업": ["구내식당/푸드코트"],
        # 기타
        "기타": [
            "농산물", "청과물", "인삼제품", "건강식품", "식품 제조", "미곡상",
            "식료품", "축산물", "담배", "건강원", "수산물", "건어물", "유제품"
        ],
    }

    # --- 2) 역매핑 ---
    reverse_map = {raw: merged for merged, raws in groups.items() for raw in raws}

    # --- 3) 매핑 적용 ---
    src = df_final["biz_type"].astype("string").str.strip()
    merged = src.map(reverse_map).fillna(src)
    merged = merged.astype("string").str.replace(r"\s+", "", regex=True).str.strip()

    # --- 4) 새로운 컬럼 biz_type2 생성 ---
    df_final["biz_type2"] = merged.astype("category")

    return df_final


df_final = unify_biz_type(df_final)
df_final[["biz_type", "biz_type2"]].head()
df_final["biz_type2"].value_counts()


biz_type2
한식                    30561
기타                    13179
커피전문점                 10788
주점업                    6740
일식음식점업                 4896
제과점                    4838
서양식음식점업                3409
치킨전문점                  2938
김밥및기타간이음식점업            2867
피자·햄버거·샌드위치및유사음식점업     2367
중식음식점업                 2279
기타외국식음식점업               866
간이음식포장판매전문점             671
기타비알코올음료점업              151
기관구내식당업                  40
Name: count, dtype: int64

- df_final에서 mapping을 위한 새로운 상권변수 생성(biz_type2)

In [8]:
#merge용으로 년도만 나와있는 year변수 생성
df_final["year"] = pd.to_datetime(df_final["ym"], errors="coerce").dt.year

# 병합
df_final = pd.merge(
    df_final,
    avg_long[["biz_type2", "year", "평균매출"]],
    on=["biz_type2", "year"],
    how="left"
)

# 매출 추정 (단위는 avg_sales와 동일 단위)
#(동일업종 매출금액 비율 * 해당년도 업종 평균 매출금액)을 통해서 구함
df_final["sales_estimate"] = df_final["평균매출"] * (df_final["rel_sales_ratio"] / 100.0)

# 조인 매칭률
matched = df_final["평균매출"].notna().mean()
print(f"[체크] 업종-연도 매칭률: {matched:.1%}")

# 샘플 확인
print(df_final[[src_col, "ym", "year", "rel_sales_ratio", "평균매출", "sales_estimate"]].head())



[체크] 업종-연도 매칭률: 100.0%
  biz_type2         ym  year  rel_sales_ratio     평균매출  sales_estimate
0    중식음식점업 2023-01-01  2023              0.5  23531.8        117.6590
1    중식음식점업 2023-02-01  2023              0.0  23531.8          0.0000
2    중식음식점업 2023-03-01  2023              0.0  23531.8          0.0000
3    중식음식점업 2023-04-01  2023              1.3  23531.8        305.9134
4    중식음식점업 2023-05-01  2023              0.0  23531.8          0.0000


In [9]:
#만들었던 변수들 제거
df_final = df_final.drop(columns=["year", "평균매출","biz_type2"], errors="ignore")


# 2. Train / Test 데이터 분할

## 1. 분할 전 변수 처리
- 가맹점 고유 속성 관련 변수들은 분할 전에 처리 가능

In [10]:
# 결측치 확인

df_final.isnull().sum()

num                         0
ym                          0
oper_month_grp              0
sales_grp                   0
txn_cnt_grp                 0
cust_cnt_grp                0
avg_pay_grp                 0
cancel_rate_grp          6632
delivery_sales_ratio    57345
rel_sales_ratio             0
M1_SME_RY_CNT_RAT           0
sales_rank_industry         0
sales_rank_market           0
share_sales_industry        0
share_sales_market      21419
male_20below_ratio       2004
male_30_ratio            2004
male_40_ratio            2004
male_50_ratio            2004
male_60plus_ratio        2004
female_20below_ratio     2004
female_30_ratio          2004
female_40_ratio          2004
female_50_ratio          2004
female_60plus_ratio      2004
revisit_ratio            1643
new_ratio                1643
resident_ratio           7327
worker_ratio             7327
floating_ratio           7327
addr                        0
name                        0
brand_cd                74865
sigungu   

- brand_cd는 동일 브랜드 매장 식별을 위한 구분 코드<br>
->num 이라는 고유 키가 있으므로 모델링에는 brand_cd 삭제
- 대신 프랜차이즈 여부 정보는 남겨두면 의미가 있으므로 is_franchise 파생변수 생성

In [11]:
#  가맹점 여부 플래그 생성
df_final["is_franchise"] = df_final["brand_cd"].notna().astype(int)

# brand_cd 컬럼 삭제
df_final.drop(columns=["brand_cd"], inplace=True)

# 확인
print("삭제 후 컬럼 존재 여부:", "brand_cd" in df_final.columns)
print(df_final[["num", "is_franchise"]].head())


삭제 후 컬럼 존재 여부: False
          num  is_franchise
0  000F03E44A             0
1  000F03E44A             0
2  000F03E44A             0
3  000F03E44A             0
4  000F03E44A             0


- close(폐업 날짜), is_closed(폐업 여부) 변수 삭제<br>
*폐업 여부나 폐업일은 아직 일어나지 않은 미래 사건이므로
모델의 입력변수로 사용하면 안 됨*

In [12]:
df_final.drop(columns=["close"], inplace=True)

In [13]:
df_final.drop(columns=["is_closed"], inplace=True)

In [14]:
df_final.shape

(86590, 39)

In [15]:
# 결측치 확인
df_final.isnull().sum()

num                         0
ym                          0
oper_month_grp              0
sales_grp                   0
txn_cnt_grp                 0
cust_cnt_grp                0
avg_pay_grp                 0
cancel_rate_grp          6632
delivery_sales_ratio    57345
rel_sales_ratio             0
M1_SME_RY_CNT_RAT           0
sales_rank_industry         0
sales_rank_market           0
share_sales_industry        0
share_sales_market      21419
male_20below_ratio       2004
male_30_ratio            2004
male_40_ratio            2004
male_50_ratio            2004
male_60plus_ratio        2004
female_20below_ratio     2004
female_30_ratio          2004
female_40_ratio          2004
female_50_ratio          2004
female_60plus_ratio      2004
revisit_ratio            1643
new_ratio                1643
resident_ratio           7327
worker_ratio             7327
floating_ratio           7327
addr                        0
name                        0
sigungu                     0
biz_type  

- addr(주소), name(상호명)은 단순 문자열로 모델링에 제외

In [16]:
df_final.drop(columns=["addr", "name"], inplace=True)

- sigungu(시군구)는 전부 '서울 성동구'라 없어도 됨

In [17]:
df_final['sigungu'].unique()

array(['서울 성동구'], dtype=object)

In [18]:
df_final.drop(columns=["sigungu"], inplace=True)

## 2. Train / Test 분할

###  신규 가게(Cold Start) 처리 전략

####  1. 기존 분할 기준에서 발생하는 문제점

일반적으로 시계열 데이터를 **기간 기준으로 Train/Test 분리**할 때  
모든 가게를 동일하게 취급하면 다음 문제가 발생한다:

| 문제 유형 | 설명 |
|------------|------|
| **① 데이터 누락 (입력부족)** | 신규 개설 가게는 과거 3개월 입력 데이터(`t-2, t-1, t`)가 존재하지 않아 모델에 투입 불가 |
| **② 분포 불일치 (distribution shift)** | Train에는 없던 “신규 가게”가 Test에만 존재하여, 모델이 한 번도 본 적 없는 유형을 예측하게 됨 |
| **③ 모델 성능 왜곡** | 신규점은 학습 불가능한 구조인데 Test MAE/RMSE 계산 시 포함되어, 모델 성능이 인위적으로 나빠짐 |
| **④ 해석 어려움** | 모델이 “실패”한 건지 “데이터가 없어서 예측 불가”한 건지 구분이 어려움 |

>  즉, **신규 가게는 Train/Test를 동일 기준으로 자르면 불공정하게 Test에만 몰리게 됨**  
> (→ 현실적으로 “Cold Start 문제”)

---

####  2. 해결 방향: 신규점(Cold Start) 구분 및 표시

- **Train**에는 과거 이력이 충분한 기존 가게만 포함  
- **Test**에는 기존 가게 + 신규 가게 모두 포함  
- **신규 가게(`is_cold_start = 1`)** 피처를 추가해 모델이 인식하도록 처리  
- 평가 단계에서 **기존점과 신규점을 별도로 리포트**

---

####  3. 기준 시점 설정과 이유

##### 🔹 기준 시점: `2024년 3월 이후 (open > 2024-03-01)`

| 이유 | 설명 |
|------|------|
| **① 입력 구조** | 모델은 "최근 3개월 → 다음 달 예측" 구조를 사용 |
| **② 입력 확보 필요** | 예를 들어 4월 예측을 하려면 1~3월 데이터가 필요함 |
| **③ 신규점 정의** | 2024-03 이후 개설된 가게는 3개월 입력이 존재하지 않음 → Cold Start |

→ 따라서  
- **기존 가게** : `open ≤ 2024-03-01`  
- **신규 가게** : `open > 2024-03-01`

---



- 가게 개설 날짜 확인

In [19]:
import pandas as pd

df = df_final.copy()
df['open'] = pd.to_datetime(df['open'])

# 전체 개설일 범위 확인
print("📅 개설일 최소:", df['open'].min().strftime('%Y-%m-%d'))
print("📅 개설일 최대:", df['open'].max().strftime('%Y-%m-%d'))

# 연·월별 개설 가게 수 집계
open_summary = (
    df.drop_duplicates('num')  # 가게당 한 번만 카운트
      .groupby(df['open'].dt.to_period('M'))['num']
      .count()
      .rename('new_stores')
      .reset_index()
)
open_summary['open'] = open_summary['open'].astype(str)
print(open_summary.tail(12))  # 최근 12개월만 보기


📅 개설일 최소: 1990-02-28
📅 개설일 최대: 2024-12-23
        open  new_stores
298  2024-01          48
299  2024-02          47
300  2024-03          40
301  2024-04          54
302  2024-05          68
303  2024-06          53
304  2024-07          47
305  2024-08          55
306  2024-09          51
307  2024-10          58
308  2024-11          46
309  2024-12          25


In [20]:
import pandas as pd

# ===== 1️. 데이터 정렬 및 날짜형 변환 =====
df = df_final.copy()
df['ym'] = pd.to_datetime(df['ym'])
df['open'] = pd.to_datetime(df['open'])
df = df.sort_values(['num', 'ym']).reset_index(drop=True)

# ===== 2️. 다음달 예측 타깃 생성 =====W
df['y_month'] = df['ym'] + pd.offsets.MonthBegin(1)
df['y_next_rank'] = df.groupby('num')['rel_sales_ratio'].shift(-1)

# ===== 3️. 신규점 플래그 추가 =====
# 기준: 2024-03 이후 개설 가게 → 신규점으로 간주
cold_start_cutoff = pd.Timestamp('2024-03-01')
df['is_cold_start'] = (df['open'] > cold_start_cutoff).astype(int)

# ===== 4️. Train / Test 분리 =====
# 타깃 월(y_month) 기준으로 분할 (미래 정보 누수 방지)
train = df[(df['y_month'] >= '2023-04-01') & (df['y_month'] <= '2024-06-30') & (df['is_cold_start'] == 0)].copy()
test  = df[(df['y_month'] >= '2024-07-01') & (df['y_month'] <= '2024-12-31')].copy()

# ===== 5️. 요약 확인 =====
def summarize(name, data):
    print(f"\n[{name}]")
    print(f"기간: {data['y_month'].min().strftime('%Y-%m')} ~ {data['y_month'].max().strftime('%Y-%m')}")
    print(f"샘플 수: {len(data):,}")
    print(f"고유 가게 수: {data['num'].nunique():,}")
    print(f"신규점 비율: {data['is_cold_start'].mean()*100:.2f}%")

summarize("Train", train)
summarize("Test", test)



[Train]
기간: 2023-04 ~ 2024-06
샘플 수: 51,928
고유 가게 수: 3,687
신규점 비율: 0.00%

[Test]
기간: 2024-07 ~ 2024-12
샘플 수: 24,036
고유 가게 수: 4,138
신규점 비율: 8.57%


# 3. 분할 후 변수 처리

## 1. Train 전처리

In [21]:
train.head()

,num,ym,oper_month_grp,sales_grp,txn_cnt_grp,cust_cnt_grp,avg_pay_grp,cancel_rate_grp,delivery_sales_ratio,rel_sales_ratio,...,floating_ratio,biz_type,market,open,프랜차이즈여부,sales_estimate,is_franchise,y_month,y_next_rank,is_cold_start
2,000F03E44A,2023-03-01,5_75-90%,6_90%초과(하위 10% 이하),6_90%초과(하위 10% 이하),6_90%초과(하위 10% 이하),6_90%초과(하위 10% 이하),NaN,NaN,0.0,...,NaN,중식-딤섬/중식만두,뚝섬,2022-02-25,0,0.0000,0,2023-04-01,1.3,0
3,000F03E44A,2023-04-01,5_75-90%,6_90%초과(하위 10% 이하),5_75-90%,5_75-90%,3_25-50%,1_상위1구간,NaN,1.3,...,0.0,중식-딤섬/중식만두,뚝섬,2022-02-25,0,305.9134,0,2023-05-01,0.0,0
4,000F03E44A,2023-05-01,5_75-90%,6_90%초과(하위 10% 이하),5_75-90%,5_75-90%,6_90%초과(하위 10% 이하),1_상위1구간,NaN,0.0,...,100.0,중식-딤섬/중식만두,뚝섬,2022-02-25,0,0.0000,0,2023-06-01,0.1,0
5,000F03E44A,2023-06-01,5_75-90%,6_90%초과(하위 10% 이하),5_75-90%,5_75-90%,6_90%초과(하위 10% 이하),1_상위1구간,NaN,0.1,...,100.0,중식-딤섬/중식만두,뚝섬,2022-02-25,0,23.5318,0,2023-07-01,0.0,0
6,000F03E44A,2023-07-01,5_75-90%,6_90%초과(하위 10% 이하),6_90%초과(하위 10% 이하),6_90%초과(하위 10% 이하),6_90%초과(하위 10% 이하),NaN,NaN,0.0,...,NaN,중식-딤섬/중식만두,뚝섬,2022-02-25,0,0.0000,0,2023-08-01,0.0,0


In [22]:
train.isnull().sum()

num                         0
ym                          0
oper_month_grp              0
sales_grp                   0
txn_cnt_grp                 0
cust_cnt_grp                0
avg_pay_grp                 0
cancel_rate_grp          3773
delivery_sales_ratio    34378
rel_sales_ratio             0
M1_SME_RY_CNT_RAT           0
sales_rank_industry         0
sales_rank_market           0
share_sales_industry        0
share_sales_market      12803
male_20below_ratio       1152
male_30_ratio            1152
male_40_ratio            1152
male_50_ratio            1152
male_60plus_ratio        1152
female_20below_ratio     1152
female_30_ratio          1152
female_40_ratio          1152
female_50_ratio          1152
female_60plus_ratio      1152
revisit_ratio             940
new_ratio                 940
resident_ratio           4181
worker_ratio             4181
floating_ratio           4181
biz_type                    0
market                  12803
open                        0
프랜차이즈여부   

### 1. 순서형 변수
- 순서형 인코딩 처리 (1~6 숫자로 변환)
- 결측치: 중간 구간(3) or 최빈값 대체

In [23]:
grp_cols = [
    'oper_month_grp', 'sales_grp', 'txn_cnt_grp', 
    'cust_cnt_grp', 'avg_pay_grp', 'cancel_rate_grp'
]

# 중간 구간(3_25-50%) 값이 있으면 그걸 우선 사용, 아니면 최빈값으로
for c in grp_cols:
    if (train[c] == '3_25-50%').any():
        fill_val = '3_25-50%'
    else:
        fill_val = train[c].mode()[0]
    train[c] = train[c].fillna(fill_val)

# 순서형 인코딩
import re

for c in grp_cols:
    # 정규표현식으로 앞의 숫자 추출 (없으면 NaN)
    train[c] = train[c].astype(str).apply(lambda x: int(re.findall(r'^\d+', x)[0]) if re.findall(r'^\d+', x) else None)


In [24]:
train.head()

,num,ym,oper_month_grp,sales_grp,txn_cnt_grp,cust_cnt_grp,avg_pay_grp,cancel_rate_grp,delivery_sales_ratio,rel_sales_ratio,...,floating_ratio,biz_type,market,open,프랜차이즈여부,sales_estimate,is_franchise,y_month,y_next_rank,is_cold_start
2,000F03E44A,2023-03-01,5,6,6,6,6,1,NaN,0.0,...,NaN,중식-딤섬/중식만두,뚝섬,2022-02-25,0,0.0000,0,2023-04-01,1.3,0
3,000F03E44A,2023-04-01,5,6,5,5,3,1,NaN,1.3,...,0.0,중식-딤섬/중식만두,뚝섬,2022-02-25,0,305.9134,0,2023-05-01,0.0,0
4,000F03E44A,2023-05-01,5,6,5,5,6,1,NaN,0.0,...,100.0,중식-딤섬/중식만두,뚝섬,2022-02-25,0,0.0000,0,2023-06-01,0.1,0
5,000F03E44A,2023-06-01,5,6,5,5,6,1,NaN,0.1,...,100.0,중식-딤섬/중식만두,뚝섬,2022-02-25,0,23.5318,0,2023-07-01,0.0,0
6,000F03E44A,2023-07-01,5,6,6,6,6,1,NaN,0.0,...,NaN,중식-딤섬/중식만두,뚝섬,2022-02-25,0,0.0000,0,2023-08-01,0.0,0


### 2. 범주형 변수(biz_type, market)

- label encoding 진행
- market 결측은 '기타' 처리 + 'is_market_missing' 플래그 변수 추가

In [25]:
print(train['biz_type'].unique()[:10])
print(train['market'].unique()[:10])


<StringArray>
['중식-딤섬/중식만두',       '요리주점',     '백반/가정식',      '커피전문점',   '한식-찌개/전골',
   '룸살롱/단란주점',      '한식-냉면',        '축산물',  '한식-단품요리일반',         '양식']
Length: 10, dtype: string
['뚝섬' '마장동' '답십리' '성수' nan '왕십리' '금남시장' '한양대' '장한평자동차' '옥수']


In [26]:
from sklearn.preprocessing import LabelEncoder

le_biz = LabelEncoder()
le_market = LabelEncoder()

train['is_market_missing'] = train['market'].isna().astype(int)
train['market'] = train['market'].fillna('기타')

train['biz_type'] = le_biz.fit_transform(train['biz_type'].astype(str))
train['market'] = le_market.fit_transform(train['market'].astype(str))

print("biz_type 매핑:", dict(zip(le_biz.classes_, le_biz.transform(le_biz.classes_))))
print("market 매핑:", dict(zip(le_market.classes_, le_market.transform(le_market.classes_))))


biz_type 매핑: {'건강식품': 0, '건강원': 1, '건어물': 2, '구내식당/푸드코트': 3, '기사식당': 4, '기타세계요리': 5, '꼬치구이': 6, '농산물': 7, '담배': 8, '도너츠': 9, '도시락': 10, '동남아/인도음식': 11, '떡/한과': 12, '떡/한과 제조': 13, '룸살롱/단란주점': 14, '마카롱': 15, '미곡상': 16, '민속주점': 17, '반찬': 18, '백반/가정식': 19, '베이커리': 20, '분식': 21, '샌드위치/토스트': 22, '수산물': 23, '스테이크': 24, '식료품': 25, '식품 제조': 26, '아이스크림/빙수': 27, '양식': 28, '와인바': 29, '와인샵': 30, '와플/크로플': 31, '요리주점': 32, '유제품': 33, '이자카야': 34, '인삼제품': 35, '일반 유흥주점': 36, '일식-덮밥/돈가스': 37, '일식-샤브샤브': 38, '일식-우동/소바/라면': 39, '일식-참치회': 40, '일식-초밥/롤': 41, '일식당': 42, '주류': 43, '주스': 44, '중식-딤섬/중식만두': 45, '중식-훠궈/마라탕': 46, '중식당': 47, '차': 48, '청과물': 49, '축산물': 50, '치킨': 51, '카페': 52, '커피전문점': 53, '탕후루': 54, '테마카페': 55, '테이크아웃커피': 56, '포장마차': 57, '피자': 58, '한식-감자탕': 59, '한식-국밥/설렁탕': 60, '한식-국수/만두': 61, '한식-냉면': 62, '한식-단품요리일반': 63, '한식-두부요리': 64, '한식-육류/고기': 65, '한식-죽': 66, '한식-찌개/전골': 67, '한식-해물/생선': 68, '한식뷔페': 69, '한정식': 70, '햄버거': 71, '호프/맥주': 72}
market 매핑: {'건대입구': 0, '금남시장': 1, '기타': 2, '답십리': 3, '동대문역

In [27]:
train.isnull().sum()

num                         0
ym                          0
oper_month_grp              0
sales_grp                   0
txn_cnt_grp                 0
cust_cnt_grp                0
avg_pay_grp                 0
cancel_rate_grp             0
delivery_sales_ratio    34378
rel_sales_ratio             0
M1_SME_RY_CNT_RAT           0
sales_rank_industry         0
sales_rank_market           0
share_sales_industry        0
share_sales_market      12803
male_20below_ratio       1152
male_30_ratio            1152
male_40_ratio            1152
male_50_ratio            1152
male_60plus_ratio        1152
female_20below_ratio     1152
female_30_ratio          1152
female_40_ratio          1152
female_50_ratio          1152
female_60plus_ratio      1152
revisit_ratio             940
new_ratio                 940
resident_ratio           4181
worker_ratio             4181
floating_ratio           4181
biz_type                    0
market                      0
open                        0
프랜차이즈여부   

### 3. y_month(타깃 월), y_next_rank(다음 달 타깃 변수)
- y_next_rank는 같은 가게 안에서 다음 달 rel_sales_ratio 값인데 마지막 달에는 다음 달 데이터가 없기 때문에 결측치가 생김

In [28]:
train[train['y_next_rank'].isnull()]

,num,ym,oper_month_grp,sales_grp,txn_cnt_grp,cust_cnt_grp,avg_pay_grp,cancel_rate_grp,delivery_sales_ratio,rel_sales_ratio,...,biz_type,market,open,프랜차이즈여부,sales_estimate,is_franchise,y_month,y_next_rank,is_cold_start,is_market_missing
3252,099CBF2425,2023-04-01,6,5,5,5,4,1,97.4,5.1,...,63,15,2019-09-25,0,1267.1358,0,2023-05-01,NaN,0,0
5507,104031A6C2,2024-05-01,6,6,6,6,6,1,NaN,0.0,...,50,6,2020-02-13,0,0.0000,0,2024-06-01,NaN,0,0
10215,1E00B3CC15,2023-05-01,6,6,5,5,6,6,0.0,0.1,...,63,3,2014-10-29,0,24.8458,0,2023-06-01,NaN,0,0
15729,2D56DC79C1,2023-05-01,6,5,4,4,4,1,95.1,31.0,...,51,2,2019-01-29,0,8171.3830,1,2023-06-01,NaN,0,1
17640,32425BE27F,2023-05-01,6,2,2,2,3,1,33.2,141.7,...,28,2,2021-07-12,0,32520.5751,0,2023-06-01,NaN,0,1
17821,32F21633BE,2023-10-01,6,3,2,2,4,3,47.3,148.1,...,63,2,2020-07-03,0,36796.6298,0,2023-11-01,NaN,0,1
18895,365914E11C,2023-09-01,6,5,3,4,5,1,NaN,27.5,...,52,2,2021-04-08,0,3092.1000,0,2023-10-01,NaN,0,1
21257,3D4F4809F0,2023-11-01,6,2,3,3,2,5,NaN,131.1,...,42,5,2022-06-22,0,38406.1383,0,2023-12-01,NaN,0,0
36916,6A2D56434E,2023-09-01,6,3,4,4,3,1,0.0,161.1,...,51,19,2016-05-09,0,42464.8323,1,2023-10-01,NaN,0,0
48432,8D7C69A199,2023-04-01,6,6,6,6,6,6,NaN,0.0,...,53,10,2019-10-28,0,0.0000,0,2023-05-01,NaN,0,0


In [29]:
before = len(train)
train = train[train['y_next_rank'].notna()].reset_index(drop=True)
after = len(train)

print(f" y_next_rank 결측 제거 완료: {before - after}행 제거됨 (현재 {after}행)")

 y_next_rank 결측 제거 완료: 21행 제거됨 (현재 51907행)


In [30]:
train.isnull().sum()

num                         0
ym                          0
oper_month_grp              0
sales_grp                   0
txn_cnt_grp                 0
cust_cnt_grp                0
avg_pay_grp                 0
cancel_rate_grp             0
delivery_sales_ratio    34367
rel_sales_ratio             0
M1_SME_RY_CNT_RAT           0
sales_rank_industry         0
sales_rank_market           0
share_sales_industry        0
share_sales_market      12796
male_20below_ratio       1152
male_30_ratio            1152
male_40_ratio            1152
male_50_ratio            1152
male_60plus_ratio        1152
female_20below_ratio     1152
female_30_ratio          1152
female_40_ratio          1152
female_50_ratio          1152
female_60plus_ratio      1152
revisit_ratio             940
new_ratio                 940
resident_ratio           4177
worker_ratio             4177
floating_ratio           4177
biz_type                    0
market                      0
open                        0
프랜차이즈여부   

### 4. 비율형/비중형 변수 처리

1. male/female/나이대 비중 변수
- 1152개가 모두 같은 행 
-> 고객 특성 결측 플래그 + 업종*월 중앙값으로 대체

In [31]:
ratio_cols_demo = [
    'male_20below_ratio','male_30_ratio','male_40_ratio','male_50_ratio','male_60plus_ratio',
    'female_20below_ratio','female_30_ratio','female_40_ratio','female_50_ratio','female_60plus_ratio'
]

# 모든 성·연령 비율이 결측인 행만 필터링
mask_all_missing = train[ratio_cols_demo].isna().all(axis=1)
missing_demo_df = train[mask_all_missing]

print(f"전체 {len(train):,}행 중 {mask_all_missing.sum():,}행이 전부 결측입니다.")
missing_demo_df.head(10)


전체 51,907행 중 1,152행이 전부 결측입니다.


,num,ym,oper_month_grp,sales_grp,txn_cnt_grp,cust_cnt_grp,avg_pay_grp,cancel_rate_grp,delivery_sales_ratio,rel_sales_ratio,...,biz_type,market,open,프랜차이즈여부,sales_estimate,is_franchise,y_month,y_next_rank,is_cold_start,is_market_missing
75,0050D68B18,2023-05-01,6,6,6,6,6,1,NaN,0.0,...,14,10,2023-05-31,0,0.0,0,2023-06-01,654.3,0,0
133,00803E9174,2024-02-01,6,6,6,6,6,6,NaN,0.0,...,28,10,2024-02-08,0,0.0,0,2024-03-01,27.0,0,0
325,0174C56C09,2023-03-01,5,6,6,6,6,1,NaN,0.0,...,25,2,2022-02-10,0,0.0,0,2023-04-01,0.0,0,1
326,0174C56C09,2023-04-01,5,6,6,6,6,1,NaN,0.0,...,25,2,2022-02-10,0,0.0,0,2023-05-01,0.0,0,1
327,0174C56C09,2023-05-01,5,6,6,6,6,1,NaN,0.0,...,25,2,2022-02-10,0,0.0,0,2023-06-01,0.0,0,1
328,0174C56C09,2023-06-01,5,6,6,6,6,1,NaN,0.0,...,25,2,2022-02-10,0,0.0,0,2023-07-01,89.9,0,1
668,02826B57BE,2023-04-01,6,6,6,6,6,1,NaN,0.0,...,50,6,2023-04-04,0,0.0,0,2023-05-01,0.0,0,0
669,02826B57BE,2023-05-01,6,6,6,6,6,1,NaN,0.0,...,50,6,2023-04-04,0,0.0,0,2023-06-01,0.0,0,0
670,02826B57BE,2023-06-01,6,6,6,6,6,1,NaN,0.0,...,50,6,2023-04-04,0,0.0,0,2023-07-01,0.0,0,0
671,02826B57BE,2023-07-01,6,6,6,6,6,1,NaN,0.0,...,50,6,2023-04-04,0,0.0,0,2023-08-01,0.0,0,0


In [32]:
import pandas as pd

# ===== 1️. 성·연령 비율 컬럼 지정 =====
ratio_cols_demo = [
    'male_20below_ratio','male_30_ratio','male_40_ratio','male_50_ratio','male_60plus_ratio',
    'female_20below_ratio','female_30_ratio','female_40_ratio','female_50_ratio','female_60plus_ratio'
]

# ===== 2️. 결측 플래그 추가 (고객 데이터 자체 없는 행 식별) =====
train['is_customer_missing'] = train[ratio_cols_demo].isna().all(axis=1).astype(int)

# ===== 3️. 1차 처리: 업종 × 월 기준 중앙값으로 결측 대체 (시계열 구조 유지) =====
train[ratio_cols_demo] = (
    train.groupby(['biz_type','ym'])[ratio_cols_demo]
         .transform(lambda x: x.fillna(x.median()))
)

# ===== 4️. 2차 처리: 여전히 남은 결측은 업종 단위 전체 중앙값으로 대체 =====
for col in ratio_cols_demo:
    train[col] = train[col].fillna(train.groupby('biz_type')[col].transform('median'))

# ===== 5️. 확인 =====
print("남은 결측치 개수")
print(train[ratio_cols_demo].isna().sum().sort_values(ascending=False))
print(f"고객 성·연령 비율 결측 처리 완료! (is_customer_missing 플래그 포함)")


남은 결측치 개수
male_20below_ratio      0
male_30_ratio           0
male_40_ratio           0
male_50_ratio           0
male_60plus_ratio       0
female_20below_ratio    0
female_30_ratio         0
female_40_ratio         0
female_50_ratio         0
female_60plus_ratio     0
dtype: int64
고객 성·연령 비율 결측 처리 완료! (is_customer_missing 플래그 포함)


In [33]:
train.isnull().sum()

num                         0
ym                          0
oper_month_grp              0
sales_grp                   0
txn_cnt_grp                 0
cust_cnt_grp                0
avg_pay_grp                 0
cancel_rate_grp             0
delivery_sales_ratio    34367
rel_sales_ratio             0
M1_SME_RY_CNT_RAT           0
sales_rank_industry         0
sales_rank_market           0
share_sales_industry        0
share_sales_market      12796
male_20below_ratio          0
male_30_ratio               0
male_40_ratio               0
male_50_ratio               0
male_60plus_ratio           0
female_20below_ratio        0
female_30_ratio             0
female_40_ratio             0
female_50_ratio             0
female_60plus_ratio         0
revisit_ratio             940
new_ratio                 940
resident_ratio           4177
worker_ratio             4177
floating_ratio           4177
biz_type                    0
market                      0
open                        0
프랜차이즈여부   

2. revisit, new ratio 변수
- 같은 행 결측치임 
-> 고객 특성 결측 플래그 + 업종*월 중앙값으로 대체

In [34]:
# 두 컬럼 모두 결측인 행 찾기
mask_behavior_missing = train[['revisit_ratio','new_ratio']].isna().all(axis=1)

# 통계 요약
print(f"전체 {len(train):,}행 중 두 컬럼 모두 결측: {mask_behavior_missing.sum():,}행")

# 샘플 출력
train.loc[mask_behavior_missing, ['num','ym','revisit_ratio','new_ratio']].head(10)


전체 51,907행 중 두 컬럼 모두 결측: 940행


,num,ym,revisit_ratio,new_ratio
75,0050D68B18,2023-05-01,NaN,NaN
133,00803E9174,2024-02-01,NaN,NaN
325,0174C56C09,2023-03-01,NaN,NaN
326,0174C56C09,2023-04-01,NaN,NaN
327,0174C56C09,2023-05-01,NaN,NaN
328,0174C56C09,2023-06-01,NaN,NaN
668,02826B57BE,2023-04-01,NaN,NaN
669,02826B57BE,2023-05-01,NaN,NaN
670,02826B57BE,2023-06-01,NaN,NaN
671,02826B57BE,2023-07-01,NaN,NaN


In [35]:
# ===== 1️. 고객 행동 비율 컬럼 지정 =====
ratio_cols_behavior = ['revisit_ratio', 'new_ratio']

# ===== 2️. 두 컬럼이 동시에 결측인 행 플래그 추가 =====
train['is_behavior_missing'] = train[ratio_cols_behavior].isna().all(axis=1).astype(int)

# ===== 3️. 업종 × 월 기준 중앙값으로 대체 =====
train[ratio_cols_behavior] = (
    train.groupby(['biz_type', 'ym'])[ratio_cols_behavior]
         .transform(lambda x: x.fillna(x.median()))
)

print("남은 고객 행동 비율 결측치 개수:")
print(train[ratio_cols_behavior].isna().sum())
print(" revisit_ratio, new_ratio 결측 처리 완료! (is_behavior_missing 플래그 포함)")


남은 고객 행동 비율 결측치 개수:
revisit_ratio    7
new_ratio        7
dtype: int64
 revisit_ratio, new_ratio 결측 처리 완료! (is_behavior_missing 플래그 포함)


In [36]:
train['is_behavior_missing'] = train[ratio_cols_behavior].isna().all(axis=1).astype(int)

3. resident_ratio, worker_ratio, floating_ratio
- 동일 행 결측치-> 고객 특성 결측 플래그 + 업종*월 중앙값으로 대체

In [37]:
mask_type_missing = train[['resident_ratio','worker_ratio','floating_ratio']].isna().all(axis=1)

print(f"전체 {len(train):,}행 중 세 컬럼 모두 결측: {mask_type_missing.sum():,}행")

train.loc[mask_type_missing, ['num','ym','resident_ratio','worker_ratio','floating_ratio']].head(10)


전체 51,907행 중 세 컬럼 모두 결측: 4,177행


,num,ym,resident_ratio,worker_ratio,floating_ratio
0,000F03E44A,2023-03-01,NaN,NaN,NaN
4,000F03E44A,2023-07-01,NaN,NaN,NaN
5,000F03E44A,2023-08-01,NaN,NaN,NaN
7,000F03E44A,2023-10-01,NaN,NaN,NaN
9,000F03E44A,2023-12-01,NaN,NaN,NaN
10,000F03E44A,2024-01-01,NaN,NaN,NaN
30,003473B465,2023-03-01,NaN,NaN,NaN
31,003473B465,2023-04-01,NaN,NaN,NaN
34,003473B465,2023-07-01,NaN,NaN,NaN
35,003473B465,2023-08-01,NaN,NaN,NaN


In [38]:
# ===== 1️. 고객 유형 비율 컬럼 지정 =====
ratio_cols_customer_type = ['resident_ratio', 'worker_ratio', 'floating_ratio']

# ===== 2️. 세 컬럼이 모두 결측인 행 플래그 추가 =====
train['is_type_missing'] = train[ratio_cols_customer_type].isna().all(axis=1).astype(int)

# ===== 3️. 업종 × 월 기준 중앙값으로 1차 대체 =====
train[ratio_cols_customer_type] = (
    train.groupby(['biz_type','ym'])[ratio_cols_customer_type]
         .transform(lambda x: x.fillna(x.median()))
)

# ===== 4️. 업종 단위 중앙값으로 2차 대체 =====
for col in ratio_cols_customer_type:
    train[col] = train[col].fillna(train.groupby('biz_type')[col].transform('median'))


# ===== 5. 확인 =====
print("남은 고객 유형 비율 결측치 개수:")
print(train[ratio_cols_customer_type].isna().sum())
print("resident_ratio, worker_ratio, floating_ratio 결측 처리 완료! (is_type_missing 플래그 포함)")


남은 고객 유형 비율 결측치 개수:
resident_ratio    0
worker_ratio      0
floating_ratio    0
dtype: int64
resident_ratio, worker_ratio, floating_ratio 결측 처리 완료! (is_type_missing 플래그 포함)


In [39]:
train['is_type_missing'] = train[ratio_cols_customer_type].isna().all(axis=1).astype(int)

In [40]:
train.isnull().sum() # 두개 남았드아아ㅏㅇ아아!!!!!!!!!ㅠㅠㅠㅠ..

num                         0
ym                          0
oper_month_grp              0
sales_grp                   0
txn_cnt_grp                 0
cust_cnt_grp                0
avg_pay_grp                 0
cancel_rate_grp             0
delivery_sales_ratio    34367
rel_sales_ratio             0
M1_SME_RY_CNT_RAT           0
sales_rank_industry         0
sales_rank_market           0
share_sales_industry        0
share_sales_market      12796
male_20below_ratio          0
male_30_ratio               0
male_40_ratio               0
male_50_ratio               0
male_60plus_ratio           0
female_20below_ratio        0
female_30_ratio             0
female_40_ratio             0
female_50_ratio             0
female_60plus_ratio         0
revisit_ratio               7
new_ratio                   7
resident_ratio              0
worker_ratio                0
floating_ratio              0
biz_type                    0
market                      0
open                        0
프랜차이즈여부   

4. delivery_sales_ratio: 배달 매출 비율
- NaN의 의미 = “배달을 하지 않는다.”
- 즉, “0%”로 처리

In [41]:
# 1️. 결측 플래그 생성
train['is_delivery_missing'] = train['delivery_sales_ratio'].isna().astype(int)

# 2️. 결측치 0으로 대체 (배달 안함)
train['delivery_sales_ratio'] = train['delivery_sales_ratio'].fillna(0)

# 3️. 확인
print("delivery_sales_ratio 남은 결측치:", train['delivery_sales_ratio'].isna().sum())
print("delivery_sales_ratio 결측 처리 완료! (is_delivery_missing 플래그 포함)")


delivery_sales_ratio 남은 결측치: 0
delivery_sales_ratio 결측 처리 완료! (is_delivery_missing 플래그 포함)


5. share_sales_market
- 상권이 존재하지 않아 값을 구하지 못함”을 플래그로 명시하고, 값은 업종×월 중앙값으로 대체

In [42]:
# 1️. 플래그 이미 존재 (is_market_missing)
# 2️. share_sales_market 결측은 업종×월 중앙값으로 대체 (값 의미 유지)
train['share_sales_market'] = (
    train.groupby(['biz_type','ym'])['share_sales_market']
         .transform(lambda x: x.fillna(x.median()))
)

# 3️. 업종 단위 중앙값으로 2차 보정 (일부 월 표본 없음 대비)
train['share_sales_market'] = train['share_sales_market'].fillna(
    train.groupby('biz_type')['share_sales_market'].transform('median')
)

# 4️. 전체 데이터 기준 평균값으로 최종 보정
median = train['share_sales_market'].median()
train['share_sales_market'] = train['share_sales_market'].fillna(median)

# 플래그는 is_market_missing으로 연결
print("남은 share_sales_market 결측치:", train['share_sales_market'].isna().sum())
print("share_sales_market 결측은 업종×월 중앙값 보정, market 부재는 NaN 유지 (is_market_missing으로 구분)")


남은 share_sales_market 결측치: 0
share_sales_market 결측은 업종×월 중앙값 보정, market 부재는 NaN 유지 (is_market_missing으로 구분)


In [43]:
train.isnull().sum()

num                     0
ym                      0
oper_month_grp          0
sales_grp               0
txn_cnt_grp             0
cust_cnt_grp            0
avg_pay_grp             0
cancel_rate_grp         0
delivery_sales_ratio    0
rel_sales_ratio         0
M1_SME_RY_CNT_RAT       0
sales_rank_industry     0
sales_rank_market       0
share_sales_industry    0
share_sales_market      0
male_20below_ratio      0
male_30_ratio           0
male_40_ratio           0
male_50_ratio           0
male_60plus_ratio       0
female_20below_ratio    0
female_30_ratio         0
female_40_ratio         0
female_50_ratio         0
female_60plus_ratio     0
revisit_ratio           7
new_ratio               7
resident_ratio          0
worker_ratio            0
floating_ratio          0
biz_type                0
market                  0
open                    0
프랜차이즈여부                 0
sales_estimate          0
is_franchise            0
y_month                 0
y_next_rank             0
is_cold_star

---

## 2. Test 전처리

In [44]:
test.head()

,num,ym,oper_month_grp,sales_grp,txn_cnt_grp,cust_cnt_grp,avg_pay_grp,cancel_rate_grp,delivery_sales_ratio,rel_sales_ratio,...,floating_ratio,biz_type,market,open,프랜차이즈여부,sales_estimate,is_franchise,y_month,y_next_rank,is_cold_start
17,000F03E44A,2024-06-01,4_50-75%,5_75-90%,5_75-90%,5_75-90%,3_25-50%,1_상위1구간,NaN,14.2,...,83.3,중식-딤섬/중식만두,뚝섬,2022-02-25,0,4308.7770,0,2024-07-01,0.7,0
18,000F03E44A,2024-07-01,4_50-75%,6_90%초과(하위 10% 이하),5_75-90%,5_75-90%,5_75-90%,1_상위1구간,77.5,0.7,...,50.0,중식-딤섬/중식만두,뚝섬,2022-02-25,0,212.4045,0,2024-08-01,0.0,0
19,000F03E44A,2024-08-01,4_50-75%,6_90%초과(하위 10% 이하),6_90%초과(하위 10% 이하),6_90%초과(하위 10% 이하),6_90%초과(하위 10% 이하),NaN,100.0,0.0,...,NaN,중식-딤섬/중식만두,뚝섬,2022-02-25,0,0.0000,0,2024-09-01,8.8,0
20,000F03E44A,2024-09-01,4_50-75%,5_75-90%,5_75-90%,5_75-90%,2_10-25%,6_상위6구간(하위1구간),15.5,8.8,...,50.0,중식-딤섬/중식만두,뚝섬,2022-02-25,0,2670.2280,0,2024-10-01,2.2,0
21,000F03E44A,2024-10-01,4_50-75%,5_75-90%,5_75-90%,5_75-90%,4_50-75%,1_상위1구간,58.0,2.2,...,33.3,중식-딤섬/중식만두,뚝섬,2022-02-25,0,667.5570,0,2024-11-01,0.3,0


In [45]:
test.isnull().sum()

num                         0
ym                          0
oper_month_grp              0
sales_grp                   0
txn_cnt_grp                 0
cust_cnt_grp                0
avg_pay_grp                 0
cancel_rate_grp          2010
delivery_sales_ratio    15932
rel_sales_ratio             0
M1_SME_RY_CNT_RAT           0
sales_rank_industry         0
sales_rank_market           0
share_sales_industry        0
share_sales_market       6005
male_20below_ratio        542
male_30_ratio             542
male_40_ratio             542
male_50_ratio             542
male_60plus_ratio         542
female_20below_ratio      542
female_30_ratio           542
female_40_ratio           542
female_50_ratio           542
female_60plus_ratio       542
revisit_ratio             436
new_ratio                 436
resident_ratio           2224
worker_ratio             2224
floating_ratio           2224
biz_type                    0
market                   6005
open                        0
프랜차이즈여부   

### 1. 순서형 변수
- 순서형 인코딩 처리 (1~6 숫자로 변환)
- 결측치: 중간 구간(3) or 최빈값 대체

In [46]:
grp_cols = [
    'oper_month_grp', 'sales_grp', 'txn_cnt_grp', 
    'cust_cnt_grp', 'avg_pay_grp', 'cancel_rate_grp'
]

# 중간 구간(3_25-50%) 값이 있으면 그걸 우선 사용, 아니면 최빈값으로
for c in grp_cols:
    if (test[c] == '3_25-50%').any():
        fill_val = '3_25-50%'
    else:
        fill_val = test[c].mode()[0]
    test[c] = test[c].fillna(fill_val)

# 순서형 인코딩
import re

for c in grp_cols:
    # 정규표현식으로 앞의 숫자 추출 (없으면 NaN)
    test[c] = test[c].astype(str).apply(lambda x: int(re.findall(r'^\d+', x)[0]) if re.findall(r'^\d+', x) else None)


In [47]:
test.head()

,num,ym,oper_month_grp,sales_grp,txn_cnt_grp,cust_cnt_grp,avg_pay_grp,cancel_rate_grp,delivery_sales_ratio,rel_sales_ratio,...,floating_ratio,biz_type,market,open,프랜차이즈여부,sales_estimate,is_franchise,y_month,y_next_rank,is_cold_start
17,000F03E44A,2024-06-01,4,5,5,5,3,1,NaN,14.2,...,83.3,중식-딤섬/중식만두,뚝섬,2022-02-25,0,4308.7770,0,2024-07-01,0.7,0
18,000F03E44A,2024-07-01,4,6,5,5,5,1,77.5,0.7,...,50.0,중식-딤섬/중식만두,뚝섬,2022-02-25,0,212.4045,0,2024-08-01,0.0,0
19,000F03E44A,2024-08-01,4,6,6,6,6,1,100.0,0.0,...,NaN,중식-딤섬/중식만두,뚝섬,2022-02-25,0,0.0000,0,2024-09-01,8.8,0
20,000F03E44A,2024-09-01,4,5,5,5,2,6,15.5,8.8,...,50.0,중식-딤섬/중식만두,뚝섬,2022-02-25,0,2670.2280,0,2024-10-01,2.2,0
21,000F03E44A,2024-10-01,4,5,5,5,4,1,58.0,2.2,...,33.3,중식-딤섬/중식만두,뚝섬,2022-02-25,0,667.5570,0,2024-11-01,0.3,0


### 2. 범주형 변수(biz_type, market)

- label encoding 진행
- market 결측은 '기타' 처리 + 'is_market_missing' 플래그 변수 추가

In [48]:
print(test['biz_type'].unique()[:10])
print(test['market'].unique()[:10])


<StringArray>
['중식-딤섬/중식만두',       '요리주점',     '백반/가정식',      '커피전문점',   '한식-찌개/전골',
   '룸살롱/단란주점',      '한식-냉면',        '축산물',  '한식-단품요리일반',         '양식']
Length: 10, dtype: string
['뚝섬' '마장동' '답십리' '성수' nan '왕십리' '금남시장' '한양대' '장한평자동차' '옥수']


In [49]:
from sklearn.preprocessing import LabelEncoder

le_biz = LabelEncoder()
le_market = LabelEncoder()

test['is_market_missing'] = test['market'].isna().astype(int)
test['market'] = test['market'].fillna('기타')

test['biz_type'] = le_biz.fit_transform(test['biz_type'].astype(str))
test['market'] = le_market.fit_transform(test['market'].astype(str))

print("biz_type 매핑:", dict(zip(le_biz.classes_, le_biz.transform(le_biz.classes_))))
print("market 매핑:", dict(zip(le_market.classes_, le_market.transform(le_market.classes_))))


biz_type 매핑: {'건강식품': 0, '건강원': 1, '건어물': 2, '구내식당/푸드코트': 3, '기사식당': 4, '기타세계요리': 5, '꼬치구이': 6, '농산물': 7, '담배': 8, '도너츠': 9, '도시락': 10, '동남아/인도음식': 11, '떡/한과': 12, '떡/한과 제조': 13, '룸살롱/단란주점': 14, '마카롱': 15, '미곡상': 16, '민속주점': 17, '반찬': 18, '백반/가정식': 19, '베이커리': 20, '분식': 21, '샌드위치/토스트': 22, '수산물': 23, '스테이크': 24, '식료품': 25, '식품 제조': 26, '아이스크림/빙수': 27, '양식': 28, '와인바': 29, '와인샵': 30, '와플/크로플': 31, '요리주점': 32, '유제품': 33, '이자카야': 34, '인삼제품': 35, '일반 유흥주점': 36, '일식-덮밥/돈가스': 37, '일식-샤브샤브': 38, '일식-우동/소바/라면': 39, '일식-참치회': 40, '일식-초밥/롤': 41, '일식당': 42, '주류': 43, '주스': 44, '중식-딤섬/중식만두': 45, '중식-훠궈/마라탕': 46, '중식당': 47, '차': 48, '청과물': 49, '축산물': 50, '치킨': 51, '카페': 52, '커피전문점': 53, '탕후루': 54, '테마카페': 55, '테이크아웃커피': 56, '포장마차': 57, '피자': 58, '한식-감자탕': 59, '한식-국밥/설렁탕': 60, '한식-국수/만두': 61, '한식-냉면': 62, '한식-단품요리일반': 63, '한식-두부요리': 64, '한식-육류/고기': 65, '한식-죽': 66, '한식-찌개/전골': 67, '한식-해물/생선': 68, '한식뷔페': 69, '한정식': 70, '햄버거': 71, '호프/맥주': 72}
market 매핑: {'건대입구': 0, '금남시장': 1, '기타': 2, '답십리': 3, '동대문역

In [50]:
test.isnull().sum()

num                         0
ym                          0
oper_month_grp              0
sales_grp                   0
txn_cnt_grp                 0
cust_cnt_grp                0
avg_pay_grp                 0
cancel_rate_grp             0
delivery_sales_ratio    15932
rel_sales_ratio             0
M1_SME_RY_CNT_RAT           0
sales_rank_industry         0
sales_rank_market           0
share_sales_industry        0
share_sales_market       6005
male_20below_ratio        542
male_30_ratio             542
male_40_ratio             542
male_50_ratio             542
male_60plus_ratio         542
female_20below_ratio      542
female_30_ratio           542
female_40_ratio           542
female_50_ratio           542
female_60plus_ratio       542
revisit_ratio             436
new_ratio                 436
resident_ratio           2224
worker_ratio             2224
floating_ratio           2224
biz_type                    0
market                      0
open                        0
프랜차이즈여부   

### 3. y_month(타깃 월), y_next_rank(다음 달 타깃 변수)
- y_next_rank는 같은 가게 안에서 다음 달 rel_sales_ratio 값인데 마지막 달에는 다음 달 데이터가 없기 때문에 결측치가 생김

In [51]:
before = len(test)
test = test[test['y_next_rank'].notna()].reset_index(drop=True)
after = len(test)

print(f" y_next_rank 결측 제거 완료: {before - after}행 제거됨 (현재 {after}행)")

 y_next_rank 결측 제거 완료: 8행 제거됨 (현재 24028행)


In [52]:
test.isnull().sum()

num                         0
ym                          0
oper_month_grp              0
sales_grp                   0
txn_cnt_grp                 0
cust_cnt_grp                0
avg_pay_grp                 0
cancel_rate_grp             0
delivery_sales_ratio    15929
rel_sales_ratio             0
M1_SME_RY_CNT_RAT           0
sales_rank_industry         0
sales_rank_market           0
share_sales_industry        0
share_sales_market       6001
male_20below_ratio        542
male_30_ratio             542
male_40_ratio             542
male_50_ratio             542
male_60plus_ratio         542
female_20below_ratio      542
female_30_ratio           542
female_40_ratio           542
female_50_ratio           542
female_60plus_ratio       542
revisit_ratio             436
new_ratio                 436
resident_ratio           2223
worker_ratio             2223
floating_ratio           2223
biz_type                    0
market                      0
open                        0
프랜차이즈여부   

### 4. 비율형/비중형 변수 처리

1. male/female/나이대 비중 변수
- 1152개가 모두 같은 행 
-> 고객 특성 결측 플래그 + 업종*월 중앙값으로 대체

In [53]:
ratio_cols_demo = [
    'male_20below_ratio','male_30_ratio','male_40_ratio','male_50_ratio','male_60plus_ratio',
    'female_20below_ratio','female_30_ratio','female_40_ratio','female_50_ratio','female_60plus_ratio'
]

# 모든 성·연령 비율이 결측인 행만 필터링
mask_all_missing = test[ratio_cols_demo].isna().all(axis=1)
missing_demo_df = test[mask_all_missing]

print(f"전체 {len(test):,}행 중 {mask_all_missing.sum():,}행이 전부 결측입니다.")
missing_demo_df.head(10)


전체 24,028행 중 542행이 전부 결측입니다.


,num,ym,oper_month_grp,sales_grp,txn_cnt_grp,cust_cnt_grp,avg_pay_grp,cancel_rate_grp,delivery_sales_ratio,rel_sales_ratio,...,biz_type,market,open,프랜차이즈여부,sales_estimate,is_franchise,y_month,y_next_rank,is_cold_start,is_market_missing
12,003473B465,2024-06-01,3,6,6,6,6,1,NaN,0.0,...,19,5,2019-05-07,0,0.0,0,2024-07-01,0.0,0,0
13,003473B465,2024-07-01,3,6,6,6,6,1,NaN,0.0,...,19,5,2019-05-07,0,0.0,0,2024-08-01,0.0,0,0
14,003473B465,2024-08-01,3,6,6,6,6,1,NaN,0.0,...,19,5,2019-05-07,0,0.0,0,2024-09-01,0.0,0,0
15,003473B465,2024-09-01,3,6,6,6,6,1,NaN,0.0,...,19,5,2019-05-07,0,0.0,0,2024-10-01,0.0,0,0
16,003473B465,2024-10-01,3,6,6,6,6,1,NaN,0.0,...,19,5,2019-05-07,0,0.0,0,2024-11-01,0.0,0,0
17,003473B465,2024-11-01,3,6,6,6,6,1,NaN,0.0,...,19,5,2019-05-07,0,0.0,0,2024-12-01,0.0,0,0
348,0305234DDB,2024-06-01,3,6,6,6,6,1,NaN,0.0,...,25,10,2017-08-30,0,0.0,0,2024-07-01,0.0,0,0
349,0305234DDB,2024-07-01,3,6,6,6,6,1,NaN,0.0,...,25,10,2017-08-30,0,0.0,0,2024-08-01,0.0,0,0
350,0305234DDB,2024-08-01,3,6,6,6,6,1,NaN,0.0,...,25,10,2017-08-30,0,0.0,0,2024-09-01,0.0,0,0
351,0305234DDB,2024-09-01,3,6,6,6,6,1,NaN,0.0,...,25,10,2017-08-30,0,0.0,0,2024-10-01,0.0,0,0


In [54]:
import pandas as pd

# ===== 1️. 성·연령 비율 컬럼 지정 =====
ratio_cols_demo = [
    'male_20below_ratio','male_30_ratio','male_40_ratio','male_50_ratio','male_60plus_ratio',
    'female_20below_ratio','female_30_ratio','female_40_ratio','female_50_ratio','female_60plus_ratio'
]

# ===== 2️. 결측 플래그 추가 (고객 데이터 자체 없는 행 식별) =====
test['is_customer_missing'] = test[ratio_cols_demo].isna().all(axis=1).astype(int)

# ===== 3️. 1차 처리: 업종 × 월 기준 중앙값으로 결측 대체 (시계열 구조 유지) =====
test[ratio_cols_demo] = (
    test.groupby(['biz_type','ym'])[ratio_cols_demo]
         .transform(lambda x: x.fillna(x.median()))
)

# ===== 4️. 2차 처리: 여전히 남은 결측은 업종 단위 전체 중앙값으로 대체 =====
for col in ratio_cols_demo:
    test[col] = test[col].fillna(test.groupby('biz_type')[col].transform('median'))

# ===== 5️. 확인 =====
print("남은 결측치 개수")
print(test[ratio_cols_demo].isna().sum().sort_values(ascending=False))
print(f"고객 성·연령 비율 결측 처리 완료! (is_customer_missing 플래그 포함)")


남은 결측치 개수
male_20below_ratio      0
male_30_ratio           0
male_40_ratio           0
male_50_ratio           0
male_60plus_ratio       0
female_20below_ratio    0
female_30_ratio         0
female_40_ratio         0
female_50_ratio         0
female_60plus_ratio     0
dtype: int64
고객 성·연령 비율 결측 처리 완료! (is_customer_missing 플래그 포함)


In [55]:
test.isnull().sum()

num                         0
ym                          0
oper_month_grp              0
sales_grp                   0
txn_cnt_grp                 0
cust_cnt_grp                0
avg_pay_grp                 0
cancel_rate_grp             0
delivery_sales_ratio    15929
rel_sales_ratio             0
M1_SME_RY_CNT_RAT           0
sales_rank_industry         0
sales_rank_market           0
share_sales_industry        0
share_sales_market       6001
male_20below_ratio          0
male_30_ratio               0
male_40_ratio               0
male_50_ratio               0
male_60plus_ratio           0
female_20below_ratio        0
female_30_ratio             0
female_40_ratio             0
female_50_ratio             0
female_60plus_ratio         0
revisit_ratio             436
new_ratio                 436
resident_ratio           2223
worker_ratio             2223
floating_ratio           2223
biz_type                    0
market                      0
open                        0
프랜차이즈여부   

2. revisit, new ratio 변수
- 같은 행 결측치임 
-> 고객 특성 결측 플래그 + 업종*월 중앙값으로 대체

In [56]:
# 두 컬럼 모두 결측인 행 찾기
mask_behavior_missing = test[['revisit_ratio','new_ratio']].isna().all(axis=1)

# 통계 요약
print(f"전체 {len(test):,}행 중 두 컬럼 모두 결측: {mask_behavior_missing.sum():,}행")

# 샘플 출력
test.loc[mask_behavior_missing, ['num','ym','revisit_ratio','new_ratio']].head(10)


전체 24,028행 중 두 컬럼 모두 결측: 436행


,num,ym,revisit_ratio,new_ratio
12,003473B465,2024-06-01,NaN,NaN
13,003473B465,2024-07-01,NaN,NaN
14,003473B465,2024-08-01,NaN,NaN
15,003473B465,2024-09-01,NaN,NaN
16,003473B465,2024-10-01,NaN,NaN
17,003473B465,2024-11-01,NaN,NaN
348,0305234DDB,2024-06-01,NaN,NaN
349,0305234DDB,2024-07-01,NaN,NaN
350,0305234DDB,2024-08-01,NaN,NaN
351,0305234DDB,2024-09-01,NaN,NaN


In [57]:
# ===== 1️. 고객 행동 비율 컬럼 지정 =====
ratio_cols_behavior = ['revisit_ratio', 'new_ratio']

# ===== 2️. 두 컬럼이 동시에 결측인 행 플래그 추가 =====
test['is_behavior_missing'] = test[ratio_cols_behavior].isna().all(axis=1).astype(int)

# ===== 3️. 업종 × 월 기준 중앙값으로 대체 =====
test[ratio_cols_behavior] = (
    test.groupby(['biz_type', 'ym'])[ratio_cols_behavior]
         .transform(lambda x: x.fillna(x.median()))
)

print("남은 고객 행동 비율 결측치 개수:")
print(test[ratio_cols_behavior].isna().sum())
print(" revisit_ratio, new_ratio 결측 처리 완료! (is_behavior_missing 플래그 포함)")


남은 고객 행동 비율 결측치 개수:
revisit_ratio    0
new_ratio        0
dtype: int64
 revisit_ratio, new_ratio 결측 처리 완료! (is_behavior_missing 플래그 포함)


In [58]:
test['is_behavior_missing'] = test[ratio_cols_behavior].isna().all(axis=1).astype(int)

3. resident_ratio, worker_ratio, floating_ratio
- 동일 행 결측치-> 고객 특성 결측 플래그 + 업종*월 중앙값으로 대체

In [59]:
mask_type_missing = test[['resident_ratio','worker_ratio','floating_ratio']].isna().all(axis=1)

print(f"전체 {len(test):,}행 중 세 컬럼 모두 결측: {mask_type_missing.sum():,}행")

test.loc[mask_type_missing, ['num','ym','resident_ratio','worker_ratio','floating_ratio']].head(10)


전체 24,028행 중 세 컬럼 모두 결측: 2,223행


,num,ym,resident_ratio,worker_ratio,floating_ratio
2,000F03E44A,2024-08-01,NaN,NaN,NaN
12,003473B465,2024-06-01,NaN,NaN,NaN
13,003473B465,2024-07-01,NaN,NaN,NaN
14,003473B465,2024-08-01,NaN,NaN,NaN
15,003473B465,2024-09-01,NaN,NaN,NaN
16,003473B465,2024-10-01,NaN,NaN,NaN
17,003473B465,2024-11-01,NaN,NaN,NaN
60,0080644746,2024-06-01,NaN,NaN,NaN
61,0080644746,2024-07-01,NaN,NaN,NaN
62,0080644746,2024-08-01,NaN,NaN,NaN


In [60]:
# ===== 1️. 고객 유형 비율 컬럼 지정 =====
ratio_cols_customer_type = ['resident_ratio', 'worker_ratio', 'floating_ratio']

# ===== 2️. 세 컬럼이 모두 결측인 행 플래그 추가 =====
test['is_type_missing'] = test[ratio_cols_customer_type].isna().all(axis=1).astype(int)

# ===== 3️. 업종 × 월 기준 중앙값으로 1차 대체 =====
test[ratio_cols_customer_type] = (
    test.groupby(['biz_type','ym'])[ratio_cols_customer_type]
         .transform(lambda x: x.fillna(x.median()))
)

# ===== 4️. 업종 단위 중앙값으로 2차 대체 =====
for col in ratio_cols_customer_type:
    test[col] = test[col].fillna(test.groupby('biz_type')[col].transform('median'))


# ===== 5. 확인 =====
print("남은 고객 유형 비율 결측치 개수:")
print(test[ratio_cols_customer_type].isna().sum())
print("resident_ratio, worker_ratio, floating_ratio 결측 처리 완료! (is_type_missing 플래그 포함)")


남은 고객 유형 비율 결측치 개수:
resident_ratio    0
worker_ratio      0
floating_ratio    0
dtype: int64
resident_ratio, worker_ratio, floating_ratio 결측 처리 완료! (is_type_missing 플래그 포함)


In [61]:
test['is_type_missing'] = test[ratio_cols_customer_type].isna().all(axis=1).astype(int)

In [62]:
test.isnull().sum() # 두개 남았드아아ㅏㅇ아아!!!!!!!!!ㅠㅠㅠㅠ..

num                         0
ym                          0
oper_month_grp              0
sales_grp                   0
txn_cnt_grp                 0
cust_cnt_grp                0
avg_pay_grp                 0
cancel_rate_grp             0
delivery_sales_ratio    15929
rel_sales_ratio             0
M1_SME_RY_CNT_RAT           0
sales_rank_industry         0
sales_rank_market           0
share_sales_industry        0
share_sales_market       6001
male_20below_ratio          0
male_30_ratio               0
male_40_ratio               0
male_50_ratio               0
male_60plus_ratio           0
female_20below_ratio        0
female_30_ratio             0
female_40_ratio             0
female_50_ratio             0
female_60plus_ratio         0
revisit_ratio               0
new_ratio                   0
resident_ratio              0
worker_ratio                0
floating_ratio              0
biz_type                    0
market                      0
open                        0
프랜차이즈여부   

4. delivery_sales_ratio: 배달 매출 비율
- NaN의 의미 = “배달을 하지 않는다.”
- 즉, “0%”로 처리

In [63]:
# 1️. 결측 플래그 생성
test['is_delivery_missing'] = test['delivery_sales_ratio'].isna().astype(int)

# 2️. 결측치 0으로 대체 (배달 안함)
test['delivery_sales_ratio'] = test['delivery_sales_ratio'].fillna(0)

# 3️. 확인
print("delivery_sales_ratio 남은 결측치:", test['delivery_sales_ratio'].isna().sum())
print("delivery_sales_ratio 결측 처리 완료! (is_delivery_missing 플래그 포함)")


delivery_sales_ratio 남은 결측치: 0
delivery_sales_ratio 결측 처리 완료! (is_delivery_missing 플래그 포함)


5. share_sales_market
- 상권이 존재하지 않아 값을 구하지 못함”을 플래그로 명시하고, 값은 업종×월 중앙값으로 대체

In [64]:
# 1️. 플래그 이미 존재 (is_market_missing)
# 2️. share_sales_market 결측은 업종×월 중앙값으로 대체 (값 의미 유지)
test['share_sales_market'] = (
    test.groupby(['biz_type','ym'])['share_sales_market']
         .transform(lambda x: x.fillna(x.median()))
)

# 3️. 업종 단위 중앙값으로 2차 보정 (일부 월 표본 없음 대비)
test['share_sales_market'] = test['share_sales_market'].fillna(
    test.groupby('biz_type')['share_sales_market'].transform('median')
)

# 4️. 전체 데이터 기준 평균값으로 최종 보정
median = test['share_sales_market'].median()
test['share_sales_market'] = test['share_sales_market'].fillna(median)

# 플래그는 is_market_missing으로 연결
print("남은 share_sales_market 결측치:", test['share_sales_market'].isna().sum())
print("share_sales_market 결측은 업종×월 중앙값 보정, market 부재는 NaN 유지 (is_market_missing으로 구분)")


남은 share_sales_market 결측치: 0
share_sales_market 결측은 업종×월 중앙값 보정, market 부재는 NaN 유지 (is_market_missing으로 구분)


In [65]:
test.isnull().sum()

num                     0
ym                      0
oper_month_grp          0
sales_grp               0
txn_cnt_grp             0
cust_cnt_grp            0
avg_pay_grp             0
cancel_rate_grp         0
delivery_sales_ratio    0
rel_sales_ratio         0
M1_SME_RY_CNT_RAT       0
sales_rank_industry     0
sales_rank_market       0
share_sales_industry    0
share_sales_market      0
male_20below_ratio      0
male_30_ratio           0
male_40_ratio           0
male_50_ratio           0
male_60plus_ratio       0
female_20below_ratio    0
female_30_ratio         0
female_40_ratio         0
female_50_ratio         0
female_60plus_ratio     0
revisit_ratio           0
new_ratio               0
resident_ratio          0
worker_ratio            0
floating_ratio          0
biz_type                0
market                  0
open                    0
프랜차이즈여부                 0
sales_estimate          0
is_franchise            0
y_month                 0
y_next_rank             0
is_cold_star

---

# 4. 슬라이딩 데이터셋 생성

In [66]:
train.columns

Index(['num', 'ym', 'oper_month_grp', 'sales_grp', 'txn_cnt_grp',
       'cust_cnt_grp', 'avg_pay_grp', 'cancel_rate_grp',
       'delivery_sales_ratio', 'rel_sales_ratio', 'M1_SME_RY_CNT_RAT',
       'sales_rank_industry', 'sales_rank_market', 'share_sales_industry',
       'share_sales_market', 'male_20below_ratio', 'male_30_ratio',
       'male_40_ratio', 'male_50_ratio', 'male_60plus_ratio',
       'female_20below_ratio', 'female_30_ratio', 'female_40_ratio',
       'female_50_ratio', 'female_60plus_ratio', 'revisit_ratio', 'new_ratio',
       'resident_ratio', 'worker_ratio', 'floating_ratio', 'biz_type',
       'market', 'open', '프랜차이즈여부', 'sales_estimate', 'is_franchise',
       'y_month', 'y_next_rank', 'is_cold_start', 'is_market_missing',
       'is_customer_missing', 'is_behavior_missing', 'is_type_missing',
       'is_delivery_missing'],
      dtype='object')

In [67]:
train.head()

,num,ym,oper_month_grp,sales_grp,txn_cnt_grp,cust_cnt_grp,avg_pay_grp,cancel_rate_grp,delivery_sales_ratio,rel_sales_ratio,...,sales_estimate,is_franchise,y_month,y_next_rank,is_cold_start,is_market_missing,is_customer_missing,is_behavior_missing,is_type_missing,is_delivery_missing
0,000F03E44A,2023-03-01,5,6,6,6,6,1,0.0,0.0,...,0.0000,0,2023-04-01,1.3,0,0,0,0,0,1
1,000F03E44A,2023-04-01,5,6,5,5,3,1,0.0,1.3,...,305.9134,0,2023-05-01,0.0,0,0,0,0,0,1
2,000F03E44A,2023-05-01,5,6,5,5,6,1,0.0,0.0,...,0.0000,0,2023-06-01,0.1,0,0,0,0,0,1
3,000F03E44A,2023-06-01,5,6,5,5,6,1,0.0,0.1,...,23.5318,0,2023-07-01,0.0,0,0,0,0,0,1
4,000F03E44A,2023-07-01,5,6,6,6,6,1,0.0,0.0,...,0.0000,0,2023-08-01,0.0,0,0,0,0,0,1


- 고정 특성은 1개만 유지, 시계열 변수만 윈도우 처리
- 각 시계열 변수(A)의 최근 3개월 평균/표준편차/마지막값 생성 -> 1차 모델링하기에 개별 월별 피처보다 요약형이 유리함

In [68]:
import pandas as pd
import numpy as np

def make_sliding_dataset(df, window=3, target_col='y_next_rank'):
    """
    최근 n개월(window) 데이터를 입력(X), 다음 달(target_col)을 출력(y)으로 하는 슬라이딩 윈도우 데이터셋 생성
    - 고정 피처(fixed_cols)는 1개 값만 유지
    - 시계열 피처(time_cols)는 최근 window개월 평균, 표준편차, 마지막값으로 요약
    """

    df = df.sort_values(['num', 'ym']).reset_index(drop=True)
    result_rows = []

    # --- 1️. 고정 변수 / 시계열 변수 구분 ---
    fixed_cols = [
        'num', 'biz_type', 'market', 'is_franchise', 'open',
        'is_cold_start', 'is_market_missing'
    ]
    exclude_cols = fixed_cols + ['ym', 'y_month', target_col]
    time_cols = [c for c in df.columns if c not in exclude_cols]

    for num, group in df.groupby('num'):
        group = group.sort_values('ym').reset_index(drop=True)
        if len(group) <= window:
            continue

        for i in range(len(group) - window):
            window_df = group.iloc[i:i+window]
            target_row = group.iloc[i+window]

            # --- 2️. 시계열 변수 요약 (평균, 표준편차, 마지막값) ---
            mean_features = window_df[time_cols].mean().add_suffix('_mean')
            std_features = window_df[time_cols].std().add_suffix('_std')
            last_features = window_df[time_cols].iloc[-1].add_suffix('_last')

            # --- 3️. 고정 변수 (변하지 않는 특성) ---
            fixed_info = target_row[fixed_cols]  # 마지막 달 기준으로 유지

            # --- 4️. 메타 정보 ---
            meta = {
                'base_ym': window_df['ym'].iloc[-1],
                'target_ym': target_row['ym'],
                'y': target_row[target_col],
            }

            row = pd.concat([fixed_info, pd.Series(meta), mean_features, std_features, last_features])
            result_rows.append(row)

    sliding_df = pd.DataFrame(result_rows)
    return sliding_df.reset_index(drop=True)


In [69]:
train_sliding = make_sliding_dataset(train, window=3, target_col='y_next_rank')

In [70]:
test_sliding = make_sliding_dataset(test, window=3, target_col='y_next_rank')

- 확인

In [71]:
print(f" 슬라이딩 데이터셋 크기: {train_sliding.shape}")
print(train_sliding.columns)   # 앞부분 컬럼 구조 확인

 슬라이딩 데이터셋 크기: (40856, 112)
Index(['num', 'biz_type', 'market', 'is_franchise', 'open', 'is_cold_start',
       'is_market_missing', 'base_ym', 'target_ym', 'y',
       ...
       'new_ratio_last', 'resident_ratio_last', 'worker_ratio_last',
       'floating_ratio_last', '프랜차이즈여부_last', 'sales_estimate_last',
       'is_customer_missing_last', 'is_behavior_missing_last',
       'is_type_missing_last', 'is_delivery_missing_last'],
      dtype='object', length=112)


In [72]:
train_sliding.head()

,num,biz_type,market,is_franchise,open,is_cold_start,is_market_missing,base_ym,target_ym,y,...,new_ratio_last,resident_ratio_last,worker_ratio_last,floating_ratio_last,프랜차이즈여부_last,sales_estimate_last,is_customer_missing_last,is_behavior_missing_last,is_type_missing_last,is_delivery_missing_last
0,000F03E44A,45,5,0,2022-02-25,0,0,2023-05-01,2023-06-01,0.0,...,33.33,0.0,0.0,100.0,0.0,0.0000,0.0,0.0,0.0,1.0
1,000F03E44A,45,5,0,2022-02-25,0,0,2023-06-01,2023-07-01,0.0,...,25.00,0.0,0.0,100.0,0.0,23.5318,0.0,0.0,0.0,1.0
2,000F03E44A,45,5,0,2022-02-25,0,0,2023-07-01,2023-08-01,0.3,...,0.00,0.0,0.0,100.0,0.0,0.0000,0.0,0.0,0.0,1.0
3,000F03E44A,45,5,0,2022-02-25,0,0,2023-08-01,2023-09-01,0.0,...,0.00,0.0,0.0,100.0,0.0,0.0000,0.0,0.0,0.0,1.0
4,000F03E44A,45,5,0,2022-02-25,0,0,2023-09-01,2023-10-01,0.0,...,20.00,0.0,0.0,100.0,0.0,70.5954,0.0,0.0,0.0,1.0


- 내보내기

In [73]:
# train 슬라이딩 데이터 저장
train_sliding.to_csv("C:/Users/eunseok/Desktop/빅콘테스트/train_sliding_sales.csv", index=False, encoding='utf-8-sig')

# test 슬라이딩 데이터 저장
test_sliding.to_csv("C:/Users/eunseok/Desktop/빅콘테스트/test_sliding_sales.csv", index=False, encoding='utf-8-sig')
